# Rotten Tomatoes Movie Data Pipeline Demo

This notebook demonstrates the complete data acquisition and cleaning pipeline for the Rotten Tomatoes movie analysis project. We'll use the automated task system to download datasets, clean the data, and scrape additional movie information from Rotten Tomatoes.

## Project Overview

This project analyzes movie reviews and ratings data from Rotten Tomatoes, combining:
- **Kaggle Dataset**: Movie critic reviews and ratings
- **Web Scraping**: Additional movie details (titles, descriptions, release years) from Rotten Tomatoes website

## Data Pipeline Steps

1. **Environment Setup**: Install dependencies and verify setup
2. **Data Download**: Download raw datasets from Kaggle
3. **Data Cleaning**: Clean and validate the downloaded data
4. **Web Scraping**: Scrape additional movie details from Rotten Tomatoes
5. **Data Exploration**: Validate and explore the final cleaned datasets

## 1. Environment Setup

First, let's verify that our environment is properly set up and all dependencies are installed.

In [1]:
# Check Python version and environment
import sys
print(f"Python version: {sys.version}")
print(f"Current working directory: {sys.path[0]}")

# Verify required packages are available
try:
    import polars as pl
    import pandas as pd
    import numpy as np
    print("✅ Core data packages available")
except ImportError as e:
    print(f"❌ Missing packages: {e}")

Python version: 3.12.0 (main, Oct  2 2023, 23:53:49) [MSC v.1929 64 bit (AMD64)]
Current working directory: C:\Users\Damsgaard\AppData\Roaming\uv\python\cpython-3.12.0-windows-x86_64-none\python312.zip
✅ Core data packages available


In [2]:
# Sync dependencies using uv
!uv sync

Resolved 127 packages in 1ms
Audited 123 packages in 0.12ms


## 2. Data Download

Now let's download the raw datasets from Kaggle. This includes the Rotten Tomatoes critic reviews dataset.

In [3]:
# Download datasets from Kaggle
# This will download and save the Rotten Tomatoes datasets locally
!task download-data

task: [sync] uv sync
Resolved 127 packages in 0.71ms
Audited 123 packages in 0.03ms
task: [download-data] uv run python src/02807_project/data_loader.py



In [5]:
# Verify downloaded data
from pathlib import Path

data_dir = Path("data/raw")
if data_dir.exists():
    files = list(data_dir.glob("*.csv"))
    print(f"📁 Found {len(files)} CSV files in {data_dir}:")
    for file in files:
        size = file.stat().st_size / 1024 / 1024  # Size in MB
        print(f"   {file.name}: {size:.2f} MB")
else:
    print("❌ Data directory not found")

📁 Found 4 CSV files in data\raw:
   actorfilms.csv: 11.81 MB
   large_movie_dataset.csv: 1561.56 MB
   rotten_tomatoes_critic_reviews.csv: 231.74 MB
   rotten_tomatoes_movie_details.csv: 6.27 MB


## 3. Data Cleaning

Next, let's clean the downloaded data by removing null values, invalid ratings, and standardizing the format.

In [6]:
# Clean the downloaded datasets
# This removes nulls, invalid ratings, and maps review scores to numeric values
!task clean-data

2025-11-28 16:48:21 | INFO | __main__:<module>:297 - 🚀 Starting data cleaning process...
2025-11-28 16:48:21 | INFO | __main__:clean_large_movie_dataset:112 - 🧹 Cleaning Large Movie Dataset...
2025-11-28 16:48:23 | INFO | __main__:clean_large_movie_dataset:158 - ✅ Saved cleaned dataset to data\clean\large_movie_dataset_clean.csv
2025-11-28 16:48:23 | INFO | __main__:clean_rotten_tomatoes_dataset:163 - 🧹 Cleaning Rotten Tomatoes Dataset...
2025-11-28 16:48:23 | INFO | __main__:clean_rotten_tomatoes_dataset:202 - ✅ Saved cleaned dataset to data\clean\rotten_tomatoes_critic_reviews_clean.csv
2025-11-28 16:48:23 | INFO | __main__:clean_rotten_tomatoes_movie_details:207 - 🧹 Cleaning Rotten Tomatoes Movie Details...
2025-11-28 16:48:23 | INFO | __main__:clean_rotten_tomatoes_movie_details:253 - ✅ Saved cleaned movie details to data\clean\rotten_tomatoes_movie_details_clean.csv
2025-11-28 16:48:23 | INFO | __main__:clean_actorfilms_dataset:258 - 🧹 Cleaning Actor Films Dataset...
2025-11-28 16

task: [clean-data] uv run python src/02807_project/data_cleaner.py



In [7]:
# Quick exploration of cleaned data
cleaned_files = list(Path("data/clean").glob("*.csv"))
if cleaned_files:
    print(f"📁 Cleaned datasets available:")
    for file in cleaned_files:
        df = pl.read_csv(file)
        print(f"\n📊 {file.name}:")
        print(f"   Rows: {len(df):,}")
        print(f"   Columns: {len(df.columns)}")
        print(f"   Columns: {', '.join(df.columns)}")
else:
    print("❌ No cleaned data files found")

📁 Cleaned datasets available:

📊 actorfilms_clean.csv:
   Rows: 191,873
   Columns: 7
   Columns: Actor, ActorID, Film, Year, Votes, Rating, FilmID

📊 large_movie_dataset_clean.csv:
   Rows: 25,000,095
   Columns: 7
   Columns: , User_Id, Movie_Name, Rating, Genre, Release_Year, Movie_Name_Normalized

📊 rotten_tomatoes_critic_reviews_clean.csv:
   Rows: 121,877
   Columns: 9
   Columns: rotten_tomatoes_link, critic_name, top_critic, publisher_name, review_type, review_score, review_date, review_content, review_score_numeric

📊 rotten_tomatoes_movie_details_clean.csv:
   Rows: 17,661
   Columns: 4
   Columns: rotten_tomatoes_link, title, description, release_year


## 4. Web Scraping - Rotten Tomatoes Data

Now let's scrape additional movie details (titles, descriptions, release years) from Rotten Tomatoes using the movie IDs from our cleaned dataset.

In [ ]:
# Scrape movie details from Rotten Tomatoes
# This will scrape titles, descriptions, and release years for movies in our dataset
# Note: This may take several minutes depending on the number of movies
!task scrape-rt-movies

## 5. Handling Failed Scrapes

If some movies failed to scrape (due to network issues, rate limiting, etc.), we can retry them specifically.

In [ ]:
# Check for failed scrapes and retry them
# This will only re-scrape movies that failed in the previous attempt
# Note: Run this only if you suspect some scrapes failed
!task retry-failed-scrapes

## 6. Final Data Exploration

Let's explore our complete dataset including the scraped Rotten Tomatoes data.

In [8]:
# Explore all datasets (raw and cleaned)
# This provides comprehensive statistics and summaries
!task explore-data

2025-11-28 16:48:37 | INFO | __main__:explore_dataset:12 - === Exploring Large Movie Dataset (Raw) ===
2025-11-28 16:48:37 | INFO | __main__:explore_dataset:80 - 📊 Dataset size: 25,000,095 total rows
2025-11-28 16:48:37 | INFO | __main__:explore_dataset:81 - 🔍 Sample size: 1000 rows x 5 columns
2025-11-28 16:48:37 | INFO | __main__:explore_dataset:82 - Basic statistics:
2025-11-28 16:48:37 | INFO | __main__:explore_dataset:83 - shape: (9, 6)
┌────────────┬────────────┬──────────┬───────────────────────┬──────────┬────────────────────┐
│ statistic  ┆            ┆ User_Id  ┆ Movie_Name            ┆ Rating   ┆ Genre              │
│ ---        ┆ ---        ┆ ---      ┆ ---                   ┆ ---      ┆ ---                │
│ str        ┆ f64        ┆ f64      ┆ str                   ┆ f64      ┆ str                │
╞════════════╪════════════╪══════════╪═══════════════════════╪══════════╪════════════════════╡
│ count      ┆ 1000.0     ┆ 1000.0   ┆ 1000                  ┆ 1000.0   ┆ 1000 

task: [explore-data] uv run python src/02807_project/explore_data.py --version both



# 6.5 Merge data

In [9]:
# Merge cleaned datasets into a single file
!task merge-data

2025-11-28 16:48:43 | INFO | __main__:merge_all_datasets:241 - 🚀 Starting dataset merge process...
2025-11-28 16:48:43 | INFO | __main__:merge_rotten_tomatoes_data:76 - 🔗 Merging Rotten Tomatoes datasets...
2025-11-28 16:48:43 | INFO | __main__:merge_rotten_tomatoes_data:136 - ✅ Merged Rotten Tomatoes data: 17661 movies with 121877 total reviews
2025-11-28 16:48:43 | INFO | __main__:prepare_large_movie_dataset:148 - 📊 Preparing Large Movie Dataset...
2025-11-28 16:48:45 | INFO | __main__:prepare_large_movie_dataset:184 - ✅ Prepared Large Movie Dataset: 54888 unique movies from 25000095 ratings
2025-11-28 16:48:45 | INFO | __main__:prepare_actorfilms_dataset:196 - 🎬 Preparing Actor Films Dataset...
2025-11-28 16:48:45 | INFO | __main__:prepare_actorfilms_dataset:234 - ✅ Prepared Actor Films Dataset: 40973 unique films from 191873 actor-film records
2025-11-28 16:48:45 | INFO | __main__:merge_all_datasets:261 - 📋 Datasets available for merge: Rotten Tomatoes, Large Movie Dataset, Actor F

task: [merge-data] uv run python src/02807_project/merge_datasets.py



In [10]:
# Quick exploration of merged data
merged_files = list(Path("data/merged").glob("*.csv"))
if merged_files:
    print(f"📁 Merged datasets available:")
    for file in merged_files:
        df = pl.read_csv(file)
        print(f"\n📊 {file.name}:")
        print(f"   Rows: {len(df):,}")
        print(f"   Columns: {len(df.columns)}")
        print(f"   Columns: {', '.join(df.columns)}")
else:
    print("❌ No merged data files found")

📁 Merged datasets available:

📊 movies_merged.csv:
   Rows: 74,853
   Columns: 29
   Columns: title_normalized, movie_title, release_year, rotten_tomatoes_link, title, description, rt_critics_list, rt_publishers_list, rt_reviews_list, rt_avg_review_score, rt_review_scores_list, rt_top_critic_count, rt_review_count, rt_release_year, lm_movie_name_original, lm_avg_rating, lm_ratings_list, lm_user_ids_list, lm_genre, lm_release_year, lm_rating_count, af_film_original, af_actors_list, af_actor_ids_list, af_year, af_rating, af_votes, af_film_id, af_actor_count


## 7. Summary and Next Steps

### What We've Accomplished

✅ **Environment Setup**: Verified Python environment and installed dependencies

✅ **Data Download**: Downloaded Rotten Tomatoes datasets from Kaggle

✅ **Data Cleaning**: Cleaned data by removing nulls and standardizing formats

✅ **Web Scraping**: Scraped additional movie details from Rotten Tomatoes

✅ **Error Handling**: Implemented retry logic for failed scrapes

✅ **Data Exploration**: Validated and explored the final datasets

### Available Data Files

After running this pipeline, you'll have:

- `data/raw/rotten_tomatoes_movies.csv` - Raw movie data from Kaggle
- `data/raw/rotten_tomatoes_critic_reviews.csv` - Raw critic reviews from Kaggle
- `data/clean/rotten_tomatoes_movies_clean.csv` - Cleaned movie data
- `data/clean/rotten_tomatoes_critic_reviews_clean.csv` - Cleaned critic reviews
- `data/raw/rotten_tomatoes_movie_details.csv` - Scraped movie details (titles, descriptions, years)

### Quick Pipeline Command

For future runs, you can use the complete pipeline in one command:
```bash
task setup-data
```

This runs: sync → download-data → clean-data all in sequence.

### Next Steps

With your data ready, you can now:
- Perform sentiment analysis on critic reviews
- Analyze the relationship between critic scores and audience ratings
- Build recommendation models based on movie features
- Visualize trends in movie ratings over time

Happy analyzing! 🎬📊

# 8 Methods

## 8.1 Frequent items

In [ ]:
# Runs frequent items analysis with min_support of 0.1 and min_confidence of 0.8
# Adjust these parameters as needed for futher analysis - may take significantly longer time with lower thresholds
!task run-frequent-items -- --min-support 0.1 --min-confidence 0.8

## 8.2 Sentence embeddings

## 8.3 Graph analysis

## 8.4 Collaborative filtering

In [ ]:
# Runs collaborative filterings with ...
!task run-collaborative-filtering